# Kaggle Competiton
- https://www.kaggle.com/competitions/2019-2nd-ml-month-with-kakr/overview
- Note : [Link](https://www.notion.so/parkjaeyoung/Kaggle-4147f4c9dd0b43e284d697c1cb6d7875?pvs=4https://www.notion.so/parkjaeyoung/Kaggle-4147f4c9dd0b43e284d697c1cb6d7875?pvs=4)

# Library 및 Data Load

In [8]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler

import xgboost as xgb
from xgboost import XGBRegressor

import lightgbm as lgb
from lightgbm import LGBMRegressor


from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline


In [9]:
pd.options.display.max_rows = 100
pd.set_option("display.max_rows", 100)

# Data Load

In [31]:
sub = pd.read_csv('sub.csv',index_col=0)
x = pd.read_csv('x.csv',index_col=0)
y = pd.read_csv('y.csv',index_col=0)   # Log Scaled
test_id = pd.read_csv('test_id.csv',index_col=0)

y = y['price_logscaled'].to_list()
test_id = test_id['id'].to_list()

In [11]:
print(x.shape)
print(len(y))
print(sub.shape)

(15035, 16)
15035
(6468, 16)


# Grid Search

1. 1차  
    - Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 800, 'subsample': 0.8}
    - Best Score: 0.0265022900709913

In [12]:
# 그리드 서치 파라미터 설정
param_grid = {
    'n_estimators': [100, 300, 500, 800],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9]
}

In [13]:
# Gradient Boosting Regressor 모델 정의
gradientboost_model = GradientBoostingRegressor()



In [15]:
# 그리드 서치 수행
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

In [16]:
grid_search.fit(x, y)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 300, 500, 800],
                         'subsample': [0.6, 0.7, 0.8, 0.9]},
             scoring='neg_mean_squared_error')

In [18]:
# 최적 파라미터와 최적 점수 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", -grid_search.best_score_)

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 800, 'subsample': 0.8}
Best Score: 0.0265022900709913


# Modeling

In [19]:
best_gbr_parameter = {
                    'learning_rate': 0.05,
                    'max_depth': 5,
                    'n_estimators': 800,
                    'subsample': 0.8
}

In [23]:
gbr_model = GradientBoostingRegressor(random_state=36 , **best_gbr_parameter)
gbr_model.fit(x,y)

GradientBoostingRegressor(learning_rate=0.05, max_depth=5, n_estimators=800,
                          random_state=36, subsample=0.8)

In [25]:
#scores = cross_val_score(gbr_model, x, y, cv=5, scoring='neg_mean_squared_error')
#scoring='neg_mean_squared_error')

#RMSE 계산
#rmse_scores = np.sqrt(-scores)

#결과 출력
#print("MSE Scores:", scores)
#print("RMSE Scores:", rmse_scores)
#print("Mean RMSE:", np.mean(rmse_scores))

# Predict

In [26]:
y_pred = gbr_model.predict(sub)

In [27]:
y_pred = np.expm1(y_pred)

In [28]:
y_pred

array([ 502562.92477534,  477381.13641238, 1321846.32551518, ...,
        480823.26227539,  334410.97097947,  461107.7416517 ])

# Make Submission

In [38]:
sub_final = pd.DataFrame(data={'id':test_id,'price':y_pred})
sub_final.to_csv('submission.csv', index=False)